In [1]:
#import libraries
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
import string
import json
import itertools 
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import re

In [24]:
def wait_for(xpath):
    try:
        element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
    except:
        return False
        print("Timed Out")
    return True

In [140]:
def stats_scraper(current_page):
    soup = BeautifulSoup(current_page, 'html.parser')
    table = soup.find_all('table')[1]
    table_rows = table.find_all('tr')

    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)
    df = pd.DataFrame(res)

    df.iat[0,0]
    clean_lists = []
    clean_lists.append(res[0][0])
    clean_lists.append(res[1][0])
    clean_lists.append(res[2][0])
    clean_lists.append(res[3][0])
    clean_lists.append(res[1][2])
    clean_lists.append(res[2][2])

    for x in range(2,len(df)):
        clean_lists.append(res[3][x])

    for x in range(1,len(df)-1):
        clean_lists.append(res[4][x])

    out = pd.DataFrame(clean_lists).transpose()

    if len(out.columns)==12:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "split_1", 10: "split_2", 11: "split_3"}, inplace=True)
    if len(out.columns)==14:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "split_1", 11: "split_2", 12: "split_3", 13: "split_4"}, inplace=True)
    if len(out.columns)==16:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "split_1", 12: "split_2", 13: "split_3", 14: "split_4", 15: "split_5"}, inplace=True)
    if len(out.columns)==18:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6",  12: "split_1", 13: "split_2", 14: "split_3", 15: "split_4", 16: "split_5", 17: "split_6"}, inplace=True)
    if len(out.columns)==20:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6", 12: "time_7",  13: "split_1", 14: "split_2", 15: "split_3", 16: "split_4", 17: "split_5", 18: "split_6", 19: "split_7"}, inplace=True)
    return out

In [141]:
driver = webdriver.Firefox()
timeout = 10
driver.get('https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2021/02/06')

wait_for("/html/body/div/div[2]/table")

upper_bound = len(driver.find_elements_by_xpath("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr/td"))    

df_list_race_results = []
df_list_race_stats = []

for race in range(2,upper_bound):
    wait_for("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr/td[%d]" %race)
    driver.find_element_by_xpath("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr/td[%d]" %race).click()
    wait_for("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr/td[%d]" %race)
    small_df_list=pd.read_html(driver.page_source)
    df_list_race_results.append(small_df_list[2])
    wait_for("//div[@class='localResults commContent']/div[@class='race_tab']/table")
    current_race_stats = stats_scraper(driver.page_source)
    df_list_race_stats.append(current_race_stats)
    
# returned raced 3 twice--the wait for is just looking for the presence of the table, not the specific table
# thats loading for the next race    

In [142]:
df_list_race_results

[    Pla.  Horse No.                     Horse      Jockey       Trainer  \
 0      1          2   DESTINE JEWELLERY(D032)   J Moreira        J Size   
 1      2          5      LITTLE THUNDER(C453)    K Teetan   A T Millard   
 2      3          8          MOON PEAKS(C403)     H W Lai      Y S Tsui   
 3      4          9           MONEYMORE(B182)    C L Chau     D A Hayes   
 4      5         10          ALLOY STAR(D053)  M Chadwick      T P Yung   
 5      6          1             ANONYMA(C496)    N Callan  D E Ferraris   
 6      7          4    ALL BEST FRIENDS(A217)    Z Purton       K L Man   
 7      8         11    KWAI CHUNG ELITE(C173)   T Piccone      R Gibson   
 8      9          6           CRUCIATUS(C206)     A Badel     C W Chang   
 9     10          7  LAOS ENLIGHTENMENT(C291)      H T Mo          L Ho   
 10    11         12   JIMSON THE DRAGON(C017)      C Wong       C H Yip   
 11    12          3     WINNING STRIKES(D104)    V Borges  P O'Sullivan   
 
     Act. 

In [143]:
df_list_race_stats

[        race_id                     class         race_name       stakes  \
 0  RACE 1 (402)  Class 5 - 1200M - (40-0)  KAM TIN HANDICAP  HK$ 750,000   
 
   going         track_type   time_1   time_2     time_3 split_1 split_2  \
 0  GOOD  TURF - "C" Course  (24.18)  (46.61)  (1:10.09)   24.18   22.43   
 
   split_3  
 0   23.48  ,
         race_id                      class        race_name         stakes  \
 0  RACE 2 (403)  Class 4 - 1000M - (60-40)  MAI PO HANDICAP  HK$ 1,000,000   
 
   going         track_type   time_1   time_2   time_3 split_1 split_2 split_3  
 0  GOOD  TURF - "C" Course  (13.07)  (33.76)  (56.92)   13.07   20.69   23.16  ,
         race_id                       class           race_name  \
 0  RACE 3 (404)  Class 2 - 1600M - (100-80)  YUEN LONG HANDICAP   
 
           stakes going         track_type   time_1   time_2     time_3  \
 0  HK$ 2,200,000  GOOD  TURF - "C" Course  (25.05)  (47.50)  (1:10.82)   
 
   split_1 split_2 split_3  
 0   25.05   22.45   

In [28]:
df_list_all=pd.read_html(driver.page_source)

In [32]:
df_list_all[1]

,RACE 1 (402),RACE 1 (402).1,RACE 1 (402).2,RACE 1 (402).3,RACE 1 (402).4,RACE 1 (402).5,RACE 1 (402).6,RACE 1 (402).7,RACE 1 (402).8,RACE 1 (402).9,RACE 1 (402).10,RACE 1 (402).11,RACE 1 (402).12,RACE 1 (402).13,RACE 1 (402).14,RACE 1 (402).15,Unnamed: 16,Unnamed: 17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Class 5 - 1200M - (40-0),Going :,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,GOOD,NaN,NaN
2,KAM TIN HANDICAP,Course :,"TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course","TURF - ""C"" Course",NaN,NaN
3,"HK$ 750,000",Time :,(24.18),(46.61),(1:10.09),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Sectional Time :,24.18,22.43,23.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
df.iat[0,0]
clean_lists = []

clean_lists.append(res[0][0])
clean_lists.append(res[1][0])
clean_lists.append(res[2][0])
clean_lists.append(res[3][0])
clean_lists.append(res[1][2])
clean_lists.append(res[2][2])

for x in range(2,len(df)):
    clean_lists.append(res[3][x])

for x in range(1,len(df)-1):
    clean_lists.append(res[4][x])

out = pd.DataFrame(clean_lists).transpose()

if len(out.columns)==12:
    out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "split_1", 10: "split_2", 11: "split_3"}, inplace=True)
if len(out.columns)==14:
    out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "split_1", 11: "split_2", 12: "split_3", 13: "split_4"}, inplace=True)
if len(out.columns)==16:
    out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5" 11: "split_1", 12: "split_2", 13: "split_3", 14: "split_4", 15: "split_5"}, inplace=True)
if len(out.columns)==18:
    out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6",  12: "split_1", 13: "split_2", 14: "split_3", 15: "split_4", 16: "split_5", 17: "split_6"}, inplace=True)
if len(out.columns)==20:
    out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6", 12: "time_7",  13: "split_1", 14: "split_2", 15: "split_3", 16: "split_4", 17: "split_5", 18: "split_6", 19: "split_7"}, inplace=True)


In [135]:
out

,race_id,class,race_name,stakes,going,track_type,time_1,time_2,time_3,split_1,split_2,split_3
0,RACE 1 (402),Class 5 - 1200M - (40-0),KAM TIN HANDICAP,"HK$ 750,000",GOOD,"TURF - ""C"" Course",(24.18),(46.61),(1:10.09),24.18,22.43,23.48


In [104]:
for x in range(6,6+int((len(out.columns) - 6) / 2)):
    print(x)
#     out.rename(columns={out.columns[x]: "split_[%x]" % int(((len(out.columns) - 6) / 2)-5)}, inplace=True)
    out.rename(columns={out.columns[x]: "split_[%x]" % int(x-3)}, inplace=True)    
    
    
    cols = []
    count = 1
    for column in df.columns:
        if column == 'Goods':
            cols.append(f'Goods_{count}')
            count+=1
            continue
        cols.append(column)
    df.columns = cols

6
7
8


In [137]:
big_df_list

[    Pla.  Horse No.                     Horse      Jockey       Trainer  \
 0      1          2   DESTINE JEWELLERY(D032)   J Moreira        J Size   
 1      2          5      LITTLE THUNDER(C453)    K Teetan   A T Millard   
 2      3          8          MOON PEAKS(C403)     H W Lai      Y S Tsui   
 3      4          9           MONEYMORE(B182)    C L Chau     D A Hayes   
 4      5         10          ALLOY STAR(D053)  M Chadwick      T P Yung   
 5      6          1             ANONYMA(C496)    N Callan  D E Ferraris   
 6      7          4    ALL BEST FRIENDS(A217)    Z Purton       K L Man   
 7      8         11    KWAI CHUNG ELITE(C173)   T Piccone      R Gibson   
 8      9          6           CRUCIATUS(C206)     A Badel     C W Chang   
 9     10          7  LAOS ENLIGHTENMENT(C291)      H T Mo          L Ho   
 10    11         12   JIMSON THE DRAGON(C017)      C Wong       C H Yip   
 11    12          3     WINNING STRIKES(D104)    V Borges  P O'Sullivan   
 
     Act. 

In [ ]:
def stats_scraper(current_page)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find_all('table')[1]
    table_rows = table.find_all('tr')

    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)
    df = pd.DataFrame(res)

    df.iat[0,0]
    clean_lists = []
    clean_lists.append(res[0][0])
    clean_lists.append(res[1][0])
    clean_lists.append(res[2][0])
    clean_lists.append(res[3][0])
    clean_lists.append(res[1][2])
    clean_lists.append(res[2][2])

    for x in range(2,len(df)):
        clean_lists.append(res[3][x])

    for x in range(1,len(df)-1):
        clean_lists.append(res[4][x])

    out = pd.DataFrame(clean_lists).transpose()

    if len(out.columns)==12:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "split_1", 10: "split_2", 11: "split_3"}, inplace=True)
    if len(out.columns)==14:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "split_1", 11: "split_2", 12: "split_3", 13: "split_4"}, inplace=True)
    if len(out.columns)==16:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5" 11: "split_1", 12: "split_2", 13: "split_3", 14: "split_4", 15: "split_5"}, inplace=True)
    if len(out.columns)==18:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6",  12: "split_1", 13: "split_2", 14: "split_3", 15: "split_4", 16: "split_5", 17: "split_6"}, inplace=True)
    if len(out.columns)==20:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6", 12: "time_7",  13: "split_1", 14: "split_2", 15: "split_3", 16: "split_4", 17: "split_5", 18: "split_6", 19: "split_7"}, inplace=True)
